# 🧪 Welcome to IoTControl Simulation

This environment is ready to run Python control system simulations.

✅ Your code has already been copied to your clipboard.

👉 In the cell below, **paste your code** (`Ctrl+V` or `Cmd+V`) and press **Shift + Enter** to run it.

Enjoy!

In [ ]:
# Paste your code below and press Shift + Enter